*****************************************************
# The Social Web

## 2015-2016 Master Information Sciences

* Instructors: Davide Ceolin, Dayana Spagnuelo
* Lab Assistants: Michael Accetto, Sarthak Gupta, Ayesha Noorain 
* Exercises for Hands-on session 4
* 5 March 2020 11:00 - 12:45
* NU-5B-21, NU-6A-04, NU-6B-20, NU-6C-39, NU-6C-40   
*****************************************************

Required packages:
* feedparser, praw


In [1]:
import sys

!{sys.executable} -m pip install feedparser
!{sys.executable} -m pip install praw 


For Exercises 1-3 you will need to save the script recommendations.py (from Programming Collaborative Intelligence) :

In [2]:
# -*- coding: utf-8 -*-

#!/usr/bin/python
# -*- coding: utf-8 -*-
from math import sqrt
# A dictionary of movie critics and their ratings of a small set of movies
critics = {
    'Lisa Rose': {
        'Lady in the Water': 2.5,
        'Snakes on a Plane': 3.5,
        'Just My Luck': 3.0,
        'Superman Returns': 3.5,
        'You, Me and Dupree': 2.5,
        'The Night Listener': 3.0,
    },
    'Gene Seymour': {
        'Lady in the Water': 3.0,
        'Snakes on a Plane': 3.5,
        'Just My Luck': 1.5,
        'Superman Returns': 5.0,
        'The Night Listener': 3.0,
        'You, Me and Dupree': 3.5,
    },
    'Michael Phillips': {
        'Lady in the Water': 2.5,
        'Snakes on a Plane': 3.0,
        'Superman Returns': 3.5,
        'The Night Listener': 4.0,
    },
    'Claudia Puig': {
        'Snakes on a Plane': 3.5,
        'Just My Luck': 3.0,
        'The Night Listener': 4.5,
        'Superman Returns': 4.0,
        'You, Me and Dupree': 2.5,
    },
    'Mick LaSalle': {
        'Lady in the Water': 3.0,
        'Snakes on a Plane': 4.0,
        'Just My Luck': 2.0,
        'Superman Returns': 3.0,
        'The Night Listener': 3.0,
        'You, Me and Dupree': 2.0,
    },
    'Jack Matthews': {
        'Lady in the Water': 3.0,
        'Snakes on a Plane': 4.0,
        'The Night Listener': 3.0,
        'Superman Returns': 5.0,
        'You, Me and Dupree': 3.5,
    },
    'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0,
             'Superman Returns': 4.0},
}


def sim_distance(prefs, p1, p2):
    '''
    Returns a distance-based similarity score for person1 and person2.
    '''

    # Get the list of shared_items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they have no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[p1][item] - prefs[p2][item], 2) for item in
                         prefs[p1] if item in prefs[p2]])
    return 1 / (1 + sum_of_squares)


def sim_pearson(prefs, p1, p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2.
    '''

    # Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they are no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Sum calculations
    n = len(si)
    # Sums of all the preferences
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])
    # Sums of the squares
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])
    # Sum of the products
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])
    # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / n
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0
    r = num / den
    return r


def topMatches(
    prefs,
    person,
    n=5,
    similarity=sim_pearson,
):
    '''
    Returns the best matches for person from the prefs dictionary. 
    Number of results and similarity function are optional params.
    '''

    scores = [(similarity(prefs, person, other), other) for other in prefs
              if other != person]
    scores.sort()
    scores.reverse()
    return scores[0:n]


def getRecommendations(prefs, person, similarity=sim_pearson):
    '''
    Gets recommendations for a person by using a weighted average
    of every other user's rankings
    '''

    totals = {}
    simSums = {}
    for other in prefs:
    # Don't compare me to myself
        if other == person:
            continue
        sim = similarity(prefs, person, other)
    # Ignore scores of zero or lower
        if sim <= 0: 
            continue
        for item in prefs[other]:
            # Only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item] == 0:
                # Similarity * Score
                    totals.setdefault(item, 0)
                    # The final score is calculated by multiplying each item by the
                    #   similarity and adding these products together
                    totals[item] += prefs[other][item] * sim
                    # Sum of similarities
                    simSums.setdefault(item, 0)
                    simSums[item] += sim
    # Create the normalized list
    rankings = [(total / simSums[item], item) for (item, total) in
                totals.items()]
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings


def transformPrefs(prefs):
    '''
    Transform the recommendations into a mapping where persons are described
    with interest scores for a given title e.g. {title: person} instead of
    {person: title}.
    '''

    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            # Flip item and person
            result[item][person] = prefs[person][item]
    return result

**Exercise 1: Finding Similar Users**

In recommendations.py you see two different similarity metrics: Euclidean distance and the Pearson correlation. If you've done an Information Retrieval course, you've probably heard of these. If not, look them up.

To find similar users in your little movie data set, you can fire up the following commands in your python interpreter:

In [3]:
import recommendations # make sure recommendations.py is in the directory you're working in now

# get the distance between 'Lisa Rose' and 'Gene Seymour'
recommendations.sim_distance(recommendations.critics,'Lisa Rose','Gene Seymour') 

0.14814814814814814

Try this with other names so you can see who is closer or further.

A slightly more sophisticated distance measure is the Pearson correlation. You can find the distance between 'Lisa Rose' and 'Gene Seymour' through:

In [4]:
recommendations.sim_pearson(recommendations.critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

You can also rank people according to how close they are. For this you have the topMatches function in recommendations.py

In [5]:
recommendations.topMatches(recommendations.critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

*****************************************************
#### Task: Create a topMatches function that uses the Euclidean distance. Try it with other names and see if you find differences between the two measures, do you get different rankings with Euclidean?
*****************************************************

**Exercise 2: Recommending Items**

Finding similar persons is interesting, but of course you're more interested in finding interesting movies for a user. This is what the getRecommendations function does. For example, get recommendations for 'Toby' you can call: 

In [6]:
recommendations.getRecommendations(recommendations.critics,'Toby')
recommendations.getRecommendations(recommendations.critics,'Toby', similarity=recommendations.sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

Note that the output does not only consist of a movie title, but also a guess at what the user's rating for each movie would be.

*****************************************************
#### Task: Can you also find out how to give information on how the recommendation is built up. For example about the 'closest' person that also watched this movie?
*****************************************************


**Exercise 3: In Exercise 2, you have been building recommendations based on similar users, but you could of course also build recommendations based on similar items. In this exercise you will do this.** 

The function is essentially the same, but you need to transfer your data, from:

<code>{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}</code>

to

<code>{'Lady in the Water': {'Lisa Rose': 2.5,'Gene Seymour': 3.0},
'Snakes on a Plane': {'Lisa Rose': 3.5,'Gene Seymour': 3.5}}</code>

This is what the transformPrefs function does. 

You can now create a dictionary for movies with their scores assigned by different people by invoking:

In [7]:
movies=recommendations.transformPrefs(recommendations.critics)

And find similar items for a particular movie like this:

In [8]:
recommendations.topMatches(movies,'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

Or find people who may like a particular movie:

In [9]:
recommendations.getRecommendations(movies,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

*****************************************************
#### Task: Try to follow exactly what is going on in the last call. Notice that Michael and Jack did not rate 'Just my Luck'. How is their rating for it built up?
*****************************************************

**Exercise 4: Building a Reddit Recommender (adapted from Programming Collective Intelligence by Toby Segaran)**

Save the following code as redditrec.py after you replace the '???' in the user_agent string with your name (or any unique string).

NOTE: install praw v. 3.5 

In [11]:
# -*- coding: utf-8 -*-

import praw
import time



r = praw.Reddit(user_agent='android:com.example.myredditapp:v1.2.3 (by u/sg_ironfist)', client_id='vGNoMhBTUILWCQ',
                      client_secret='2Yj2aLeMmi54Pfi-iG0rQip3yDQ',
                      redirect_url='https://www.reddit.com/prefs/apps/'
                                   'authorize_callback')

def initializeUserDict(subreddit, count=10):
  user_dict={}
  # get the top count' popular posts
  for p1 in r.subreddit(subreddit).top(limit=count):
    # find all users who commented in this
    flat_comments = praw.helpers.flatten_tree(p1.comments)
    for p2 in flat_comments:
        try:
            user=p2.author.name
            user_dict[user]={}
        except AttributeError:
            pass
  return user_dict

def fillItems(user_dict, count=100):
  all_items={}
  # Find links posted by all users
  for user in user_dict:
      # print user
      posts = r.redditor(user).comments(limit = count)
      for post in posts:
          subreddit = post.subreddit
          # print subreddit.display_name
          if subreddit.display_name in user_dict[user]:
              user_dict[user][subreddit.display_name] += 1.0
          else:
              user_dict[user][subreddit.display_name] = 1.0
          all_items[subreddit.display_name]=1
  
  # Fill in missing items with 0
  for ratings in user_dict.values():
      for item in all_items:
          if item not in ratings:
              ratings[item]=0.0
 

You can get a list of popular recent posts about programming from the programming subreddit (https://www.reddit.com/r/programming) by invoking the code below.  Don't forget to replace the '???' in the user_agent string with your name (or any unique string).

In [12]:
r = praw.Reddit(client_id='vGNoMhBTUILWCQ',client_secret='2Yj2aLeMmi54Pfi-iG0rQip3yDQ',user_agent='android:com.example.myredditapp:v1.2.3 (by u/sg_ironfist)')

subreddit = r.subreddit("programming")
for post in r.subreddit('VUAmsterdam').top(limit=10):
    print(end='\n * ')
    print(post.title)


    



 * Can anyone tell me where this is? A friend send this and said that it was in one of the VU buildings.

 * I'm quite nervous about finding accommodation.

 * Psychology Selection Test

 * Studying abroad at VU and looking for suggestions on classes to take!

 * Any success with letting the vu search for housing for you?

 * About the Master on International crimes, conflict and criminology.

 * Math bachelor

 * Spots to study


See here a list of other subreddits you can explore with this code: https://www.reddit.com/reddits/

To automatically create a data set of reddit users similar to the movie watchers you can invoke the initializeUserDict function in redditrec.py 

In [13]:
import redditrec
from redditrec import *
redusers=initializeUserDict('programming') # or for any other subreddit

AttributeError: module 'praw' has no attribute 'helpers'

Now initializeUserDict has only created the user keys. We of course also want to know what subreddits they posted comments on. You can pull those in through:

In [14]:
fillItems(redusers)

NameError: name 'redusers' is not defined

This script may take a few minutes to collect all the data. Use this time to review what is going on in the code. Notice that users don't give ratings to subreddits, instead we are counting how many comments they posted in each subreddit. 

To recommend a similar user, we can use our first script (recommendations.py) again.

First choose a random user for whom you're going to find neighbours

In [36]:
import random
user=redusers.keys()[random.randint(0,len(redusers)-1)] 
print (user) # print the username 
recommendations.topMatches(redusers,user) # from all redditors, get the most similar to user

NameError: name 'redusers' is not defined

*****************************************************
#### Task 4: Recommend subreddits for a user based on what subreddits similar users have commented in. Recommend posts for a user based on posts they have commented on. 
*****************************************************

*****************************************************
#### Task 5: Reuse the data and scripts you have created in hands-on session 3 to build a recommender for your Facebook friends. Can you get page recommendations from your friends? What other things would be interesting to recommend?
*****************************************************

For those who are considering using recommendations further:
So far, we have been basing recommendations on the full data set, obviously this is a bad idea if you have a big data set. You can prune your dataset by first computing similar items. The code to do so is already in recommendations.py and you can work with it as follows:

In [16]:
itemsim=recommendations.calculateSimilarItems(recommendations.critics)
recommendations.getRecommendedItems(recommendations.critics,itemsim,'Toby')

AttributeError: module 'recommendations' has no attribute 'calculateSimilarItems'

You can use the code from the Computational Intelligence book on “Building the Item Comparison Dataset” (pages 23-25), https://github.com/VU-Amsterdam-Web-Media-Group/TheSocialWeb2016/blob/master/lib/Handson4-Building-Item-Comparison-Dataset.pdf